In [1]:
from dotenv import load_dotenv
import openai
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
from IPython.display import Markdown, display
from groq import Groq

In [2]:
'''pushover example
This example shows how to send a message with an attachment using the Pushover API.
import requests
r = requests.post("https://api.pushover.net/1/messages.json", data = {
  "token": "APP_TOKEN",
  "user": "USER_KEY",
  "message": "hello world"
},
files = {
  "attachment": ("image.jpg", open("your_image.jpg", "rb"), "image/jpeg")
})
print(r.text)
'''

'pushover example\nThis example shows how to send a message with an attachment using the Pushover API.\nimport requests\nr = requests.post("https://api.pushover.net/1/messages.json", data = {\n  "token": "APP_TOKEN",\n  "user": "USER_KEY",\n  "message": "hello world"\n},\nfiles = {\n  "attachment": ("image.jpg", open("your_image.jpg", "rb"), "image/jpeg")\n})\nprint(r.text)\n'

In [44]:
load_dotenv(override=True)
#openai = OpenAI()

True

In [45]:
# Load the API keys from environment variables
groq_api_key = os.getenv('GROQ_API_KEY')
if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")


deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

Groq API Key exists and begins gsk_
DeepSeek API Key exists and begins sk-


In [5]:
# For pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [6]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)


In [7]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
request = "what is the tool calling format that you use?"
messages = [{"role": "user", "content": request}]

In [ ]:
#groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
#groq = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")
#model_name = "deepseek-chat"
groq = Groq(api_key=groq_api_key)
model_name = "llama-3.3-70b-versatile"
#model_name = "qwen/qwen3-32b"
#model_name = "llama-3.1-8b-instant"
#model_name ="meta-llama/llama-4-scout-17b-16e-instruct"
#model_name = 'gemma2-9b-it'
response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))

<think>
Okay, the user is asking about the tool calling format I use. First, I need to recall the specific format of the tools I use. As Qwen, my internal structure might involve various functions and modules, but the user is probably interested in how developers or users can interact with me through an API or similar interface.

Then, the user might be a developer who wants to integrate me into their system, or someone looking to understand how my tools parse requests. I should explain the format clearly but also avoid overly technical details that could confuse non-technical users.

The main content should cover the general structure of tool calls, such as specifying parameters like the API endpoint, method, headers, and body format. It's important to provide an example, like a JSON example, so users can see a practical application. Additionally, I should mention how to handle responses and possible status codes, which helps users troubleshoot.

Wait, but the user might not be familiar with JSON structure. Including a simple example would be more helpful. Also, I should mention the different types of parameters they might need to pass, such as tokens or content type headers. I need to ensure that the explanation is concise but comprehensive.

Another consideration: the user might have existing systems and wants compatibility. So highlighting the use of standard HTTP methods (POST, GET) and JSON format is good, as these are common in APIs. Also, note any required parameters or headers, like authentication tokens or content types.

Finally, I should encourage the user to check the official documentation for the most accurate and detailed information, as the specifics might vary depending on the deployment or integration method. This helps ensure they have the most up-to-date info and avoids any outdated information from me.
</think>

The tool calling format I use to interact with external tools (e.g., APIs, plugins) is typically based on a **structured JSON** input format. This allows me to specify the tool's name, parameters, and other metadata in a standardized way. Here's the general structure:

---

### **Tool Call Format (JSON Example)**
```json
{
  "name": "tool_name",                // Name/identifier of the tool
  "arguments": {                     // Parameters to pass to the tool
    "key1": "value1",
    "key2": "value2"
  },
  "toolbench_rapidapi_key": "your-api-key" // Optional: if the tool requires authentication
}
```

---

### **Key Components**
1. **`name`**:  
   The specific tool or API endpoint being called (e.g., `"weather_api"`, `"search_engine"`).

2. **`arguments`**:  
   A dictionary of parameters required by the tool (e.g., location, query, API-specific inputs).

3. **Authentication**:  
   If the tool requires it, include the API key or token (e.g., `toolbench_rapidapi_key`).

---

### **Example Usage**
If you ask for weather information:
- **Query**: `"What's the weather in San Francisco?"`
- **Tool Call**:  
  ```json
  {
    "name": "weather_api",
    "arguments": {
      "location": "San Francisco"
    },
    "toolbench_rapidapi_key": "your-weather-api-key"
  }
  ```

---

### **Response Handling**
After executing the tool call, I receive a **JSON response** from the tool and use it to generate a natural language reply to the user.

---

### **Important Notes**
- **No Live API Calls**: In some environments (e.g., public demos), I may *simulate* tool calls for demonstration purposes.
- **Private Tools**: For security, actual API keys or sensitive data are not logged or stored in public interactions.
- **Flexibility**: The format adapts to different tools and their specific parameter requirements.

For details about integrating with specific tools (e.g., custom APIs), refer to the official Qwen documentation or the tool's API spec. Let me know if you'd like help with a specific use case! 🚀

In [107]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [108]:
def record_unanswerable_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [109]:
def nop():
    print("nop operation")

In [110]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}



In [111]:
record_unanswerable_question_json = {
        "name": "record_unanswerable_question",
        "description": "Precisely record questions that cannot and must not be answered",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "Exact text of the question"
                }
            },
            "required": ["question"],
            "additionalProperties": False
        }
    }

In [112]:
nop_json = {
        "name": "nop",
        "description": "do nothing",
        "parameters": {},
        "additionalProperties": False
    }

In [113]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unanswerable_question_json}]

#        {"type": "function", "function": nop_json}]

In [114]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        print(f"Arguments: {arguments}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [115]:
name = os.getenv("MY_NAME")
name

'Antonio Pico'

In [116]:
with open("data/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

with open("data/resume.md", "r", encoding="utf-8") as f:
    resume = f.read()

print(f"Summary: {summary[:100]}...")  # Print first 100 characters for brevity
print(f"Resume: {resume[:100]}...")  # Print first 100 characters for brevity

Summary: Computer Science PhD with experience in machine learning, deep learning, and embedded systems. Curre...
Resume: # Skills

**General:** Machine learning, Deep learning, Data Science, Electronics, Embedded Systems,...


In [117]:
system_prompt = f"""You are {name}. You are answering questions about yourself on your professional website.

## STRICT RULES:
1. NEVER invent, assume, or extrapolate information not explicitly written in your summary or resume.
2. ONLY answer questions with facts that can be directly quoted or paraphrased from your provided materials.
3. If asked about strengths/weaknesses, future plans, soft skills, or another kind of questions NOT explicitly mentioned in your materials, USE the record_unanswerable_question tool to save the question and politely decline.
4. For ANY question about information not directly stated, respond: "I don't have that specific information to share right now. Feel free to reach out to me directly via email if you'd like to discuss this further." You can vary it a little so you do not sound robotic.
5.- Try to keep the person engaged in the conversation, only say good bye when they say it first.


## CONTACT COLLECTION (BE SUBTLE):
- Only suggest contact exchange when users explicitly mention opportunities, collaborations, or wanting to work together.
- Don't push for contact info after every interaction.
- When appropriate, say something like: "That sounds interesting! If you'd like to discuss this further, I'd be happy to connect. What's your name and email?"
- ONLY use record_user_details when you have BOTH name and email.
- Once you have contact details, don't ask for them again in the same conversation.

## SPEAK AS YOURSELF:
- Use "I" statements: "I worked at...", "My experience includes...", "I studied..."
- Be conversational but professional.
- Don't refer to yourself in third person.

## ACCEPTABLE RESPONSES:
- Direct facts from your resume using first person.
- Information explicitly stated in your summary.
- Redirecting to email contact for detailed discussions.

## TOOLS USAGE:
- Use tool calling, not string messages!
- Use record_unanswerable_question for ANY question you cannot or must not answer.
- Use record_user_details ONLY when you have both name and email AND the user has expressed genuine business interest.
- Don't mention these tools to users.

## TOOL CALLING FORMAT:
When you need to use a tool, use the standard function calling format. Do NOT write tool calls as text like <function=name>. Use the proper function calling mechanism provided by the system.

## Your Summary:
{summary}

## Your Resume:
{resume}

"""

In [118]:
def chat(message, history):
    # Remove metadata field from messages
    # it is added by gradio but it is not compatible with grog's api
    cleaned_history = [
        {k: v for k, v in msg.items() if k in ['role', 'content']} 
        for msg in history
    ]
    messages = [{"role": "system", "content": system_prompt}] + cleaned_history + [{"role": "user", "content": message}]

    done = False
    while not done:
        print (messages[1:])  # Print the messages excluding the system prompt for clarity
        # This is the call to the LLM - see that we pass in the tools json

        try:
            response = groq.chat.completions.create(model=model_name, messages=messages, tools=tools, temperature=0.0, tool_choice="auto")
            print(response)
            finish_reason = response.choices[0].finish_reason
            
            # If the LLM wants to call a tool, we do that!
            
            if finish_reason=="tool_calls":
                print("Tool calls detected, handling them...", flush=True)
                message = response.choices[0].message
                tool_calls = message.tool_calls
                print(f"Tool calls: {response.choices[0]}", flush=True)
                results = handle_tool_calls(tool_calls)
                messages.append(message)
                messages.extend(results)
            else:
                done = True
        except Exception as e:
            print(f"An error occurred: {e}")
            return "error"
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


[{'role': 'user', 'content': 'hi there'}]
ChatCompletion(id='chatcmpl-4fc1bf7f-b04e-4c5a-9feb-bb6c3a0f1f5b', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', executed_tools=None, function_call=None, reasoning='Okay, the user said "hi there". I need to respond appropriately. Since it\'s a greeting, I should greet them back and offer assistance. Let me check the rules.\n\nRule 5 says to keep the conversation engaging and not say goodbye unless they do. So I should start with a friendly response. Maybe something like "Hello! How can I assist you today?" That\'s conversational and professional. No need to use any tools here because it\'s just a greeting. I\'ll make sure to use an "I" statement if needed later, but for now, a simple greeting is fine. Alright, that should work.\n', tool_calls=None))], created=1752875016, model='qwen/qwen3-32b', object='chat.completion', system_fingerpri